## Droplet-Segmentation Pipeline
### A compact, publication-ready notebook that
### 1) installs/loads Segment-Anything,  
### 2) offers optional image-enhancement for low-contrast scenes,  
### 3) supports interactive bbox annotation through `jupyter_bbox_widget`, and  
### 4) runs mask prediction + visualization + pickle export.
### 5) convert pkl file into droplet areas and radii



In [ ]:
# Installation (skip on a pre-configured env)
# ---------------------------------------------------------------------
# If you ship this notebook, keep the cell but comment-out the commands so CI can
# toggle them on demand. 

# !pip install -q git+https://github.com/facebookresearch/segment-anything.git
# !pip install -q jupyter_bbox_widget roboflow dataclasses-json supervision
# -----------------------------------------------------------------------------


# %% Imports & global settings
# --------------------------------------------------------------------------
import os
import json
import pickle
import base64
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import supervision as sv
from PIL import Image, ImageEnhance
from jupyter_bbox_widget import BBoxWidget
from segment_anything import sam_model_registry, SamPredictor

# Reproducible device selection
DEVICE      = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
MODEL_TYPE  = "vit_h"                   # keep in sync with downloaded weights
PROJECT_DIR = "<PROJECT_ROOT>"          # <- edit once, all other paths are derived
WEIGHTS_DIR = os.path.join(PROJECT_DIR, "weights")
CHECKPOINT  = os.path.join(WEIGHTS_DIR, "sam_vit_h_4b8939.pth")

assert os.path.isfile(CHECKPOINT), f"Checkpoint not found: {CHECKPOINT}"
print(f"Using device: {DEVICE} · checkpoint OK")

sam = sam_model_registry[MODEL_TYPE](checkpoint=CHECKPOINT).to(DEVICE)
mask_predictor = SamPredictor(sam)
# -----------------------------------------------------------------------------


# %% Utility 1 — optional contrast/edge enhancement
# --------------------------------------------------
def enhance_image(src: str, dst: str) -> None:
    """
    Basic CLAHE + sharpening. Keeps a preview for QC, writes `dst` for SAM.
    """
    im_bgr   = cv2.imread(src)
    gray     = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2GRAY)

    clahe    = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    contrast = clahe.apply(gray)

    kernel   = np.array([[0, -1, 0],[-1, 5,-1],[0, -1, 0]])
    sharp    = cv2.filter2D(contrast, -1, kernel)
    cv2.imwrite(dst, sharp)

    # quick side-by-side preview
    plt.figure(figsize=(9, 4))
    plt.subplot(1, 2, 1); plt.title("Raw"); plt.imshow(cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)); plt.axis("off")
    plt.subplot(1, 2, 2); plt.title("Enhanced"); plt.imshow(sharp, cmap="gray"); plt.axis("off")
    plt.tight_layout(); plt.show()


# %% Utility 2 — Image → base64 (for bbox widget)
# ------------------------------------------------
def encode_image(path: str) -> str:
    with open(path, "rb") as f:
        return "data:image/jpg;base64," + base64.b64encode(f.read()).decode("utf-8")


# %% Interactive bounding-box annotation
# --------------------------------------
def annotate_dataset(img_dir: str, save_json: str) -> None:
    # NEW: ensure destination directory exists
    os.makedirs(os.path.dirname(save_json), exist_ok=True)
    """
    Launches a `BBoxWidget` carousel.  
    Each submission writes incremental results to *save_json*.
    """
    imgs = sorted([f for f in os.listdir(img_dir)
                   if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp"))])
    if not imgs:
        raise RuntimeError("No images found for annotation.")

    w_bbox      = BBoxWidget()
    annotations = {}

    # Helpers -----------------------------------------------------------
    def update_view(idx: int) -> None:
        if idx < len(imgs):
            w_bbox.image  = encode_image(os.path.join(img_dir, imgs[idx]))
            w_bbox.bboxes = []
        else:
            print("✔ Annotation complete.")

    @w_bbox.on_submit
    def _submit():
        idx = _submit.idx
        annotations[imgs[idx]] = w_bbox.bboxes
        with open(save_json, "w") as fp:
            json.dump(annotations, fp, indent=4)
        _submit.idx += 1
        update_view(_submit.idx)
    _submit.idx = 0

    @w_bbox.on_skip
    def _skip():
        _submit.idx += 1
        update_view(_submit.idx)

    # Kick-off ----------------------------------------------------------
    update_view(0); display(w_bbox)   # noqa: F821  (OK inside a notebook)


# %% Segmentation + visual QC + pickle export
# -------------------------------------------
def segment_images(img_dir: str, annotation_json: str, result_pkl: str, show_plots: bool = True) -> None:
    # NEW: ensure destination directory exists
    os.makedirs(os.path.dirname(result_pkl), exist_ok=True)
    """
    Reads bbox annotations, runs SAM, overlays detections, and pickles a
    dict[image_name] ➜ supervision.Detections object
    """
    with open(annotation_json) as fp:
        annos = json.load(fp)

    droplet_detection = {}
    for name, bboxes in annos.items():
        img_path = os.path.join(img_dir, name)
        img_bgr  = cv2.imread(img_path)
        img_rgb  = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

        mask_predictor.set_image(img_rgb)

        boxes = np.array([
            [bb["x"], bb["y"], bb["x"] + bb["width"], bb["y"] + bb["height"]]
            for bb in bboxes
        ])

        masks = []
        for box in boxes:
            _m, _s, _ = mask_predictor.predict(box=box, multimask_output=True)
            if _m.size:
                masks.append(_m[np.argmax(_s)])

        masks = np.array(masks) if masks else np.empty((0,))
        det   = sv.Detections(xyxy=sv.mask_to_xyxy(masks), mask=masks)
        droplet_detection[name] = det

        if show_plots:  # visualize side-by-side
            fig = sv.plot_images_grid(
                images=[
                    sv.BoxAnnotator(color=sv.Color.red())
                      .annotate(scene=img_bgr.copy(), detections=det, skip_label=True),
                    sv.MaskAnnotator(color=sv.Color.red(), color_lookup=sv.ColorLookup.INDEX)
                      .annotate(scene=img_bgr.copy(), detections=det)
                ],
                grid_size=(1, 2),
                titles=["Source", "Segmented"],
                show=False
            )
            plt.show()

    os.makedirs(os.path.dirname(result_pkl), exist_ok=True)
    with open(result_pkl, "wb") as fp:
        pickle.dump(droplet_detection, fp)
    print(f"✔ Saved detections ➜ {result_pkl}")


# %% Example usage — adapt paths & run cells top-to-bottom
# --------------------------------------------------------
if __name__ == "__main__":
    # 1) Optional contrast enhancement
    # enhance_image("<RAW_IMAGE>", "<ENHANCED_IMAGE>")

    # 2) Annotate (run inside Jupyter)
    # annotate_dataset(
    #     img_dir = os.path.join(PROJECT_DIR, "data/raw/<EXPERIMENT>/Images_for_segmentation"),
    #     save_json = os.path.join(PROJECT_DIR, "data/processed/<EXPERIMENT>", "annotations.json")
    # )

    # 3) Batch segmentation
    segment_images(
        img_dir          = os.path.join(PROJECT_DIR, "data/raw/<EXPERIMENT>/Images_for_segmentation"),
        annotation_json  = os.path.join(PROJECT_DIR, "data/processed/<EXPERIMENT>", "annotations.json"),
        result_pkl       = os.path.join(PROJECT_DIR, "results/<EXPERIMENT>", "droplet_identification.pkl"),
        show_plots       = True
    )


In [ ]:
"""
Droplet radii post-processing
-----------------------------
Minimal version for GitHub/paper: no plots, no size classes.
"""

from pathlib import Path
import pickle
import numpy as np
import pandas as pd
from math import sqrt, pi


# ──────────────────────────────────────────────────────────────────────────────
# User I/O: adjust these or implement the stubs
# ──────────────────────────────────────────────────────────────────────────────
PKL_PATH = Path("results/droplet_identification.pkl")   # ← adjust
CSV_PATH = Path("data/processed/experiment.csv")        # ← adjust


def load_elapsed_times() -> dict[str, str]:
    """
    Return {image_name: 'HH:MM:SS'}.
    Implement this (e.g., read a CSV/JSON) instead of hard-coding a big dict.
    """
    raise NotImplementedError("TODO: supply elapsed-time mapping")


def save_results(df: pd.DataFrame, path: Path) -> None:
    """
    Persist the final dataframe.
    Replace with Parquet/Feather/SQL if you prefer.
    """
    df.to_csv(path, index=False)


# ──────────────────────────────────────────────────────────────────────────────
# Helpers
# ──────────────────────────────────────────────────────────────────────────────
def hms_to_seconds(hms: str) -> int:
    h, m, s = map(int, hms.split(":"))
    return 3600 * h + 60 * m + s


# ──────────────────────────────────────────────────────────────────────────────
# Core processing
# ──────────────────────────────────────────────────────────────────────────────
# 1. Load detections
with PKL_PATH.open("rb") as fh:
    droplet_id = pickle.load(fh)

elapsed = load_elapsed_times()                 # {img: 'HH:MM:SS'}
conv_factor = (1 / 0.877) ** 2                # px² → µm²

# 2. Sort frames numerically and build {elapsed_HH:MM:SS: np.ndarray[radius_mm]}
radii_by_time = {}
for img, det in sorted(droplet_id.items(),
                       key=lambda kv: int(kv[0].split('_')[0])):
    area_um2 = det.area * conv_factor          # np.ndarray, µm²
    radius_mm = np.sqrt(area_um2 / pi) / 1e3   # convert µm → mm
    radii_by_time[elapsed[img]] = radius_mm

# 3. Aggregate: average radius per frame
records = [
    {"Elapsed": hms_to_seconds(ts),
     "AvgRadius_mm": rs.mean()}
    for ts, rs in sorted(radii_by_time.items())
]

df_radii = pd.DataFrame(records)

# 4. Merge with existing processed CSV
df_proc  = pd.read_csv(CSV_PATH)
df_final = pd.merge_asof(df_proc.sort_values("Elapsed"),
                         df_radii.sort_values("Elapsed"),
                         on="Elapsed", direction="nearest")

# Optional: fill any gaps
df_final["AvgRadius_mm"] = df_final["AvgRadius_mm"].interpolate()

# 5. Save
save_results(df_final, CSV_PATH)              # <-- or change the path


## Plots

In [ ]:
# Surface relative humidity and droplet count growth LUNA 1M NaCl
import matplotlib.pyplot as plt
import scienceplots
import matplotlib.colors as mcolors
import pandas as pd
import numpy as np
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter

# Define the custom color palette
def get_colors():
    return {
        'dark_blue': '#082a54',
        'purple': '#6a3577',  # Darker purple
        'teal': '#2b736a',    # Darker teal
        'gold': '#b89538',
        'red': '#e02b35',
        'dark_gray': '#1f77b4'  # Darker gray
    }

# Get the colors
colors = get_colors()

# Use custom style
plt.style.use('default')
plt.style.use(['science'])

df = pd.read_csv("path_to_csv")

# Set 'Elapsed' as the index
df.set_index('Elapsed', inplace=True)

# Function to format x-axis as H:M
def format_time(seconds, pos):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    return f"{hours:02}:{minutes:02}"

# Apply a moving average to smooth the data
window_size_main = 1  # Window size for the first subplot
window_size_droplets = 1  # Window size for the second subplot
df['Surface Temp (°C)_smooth'] = df['Surface Temp (°C) '].rolling(window=window_size_main).mean()
df['Temperature (°C)_smooth'] = df['Temperature (°C)'].rolling(window=window_size_main).mean()
df['Relative Humidity (%)_smooth'] = df['Relative Humidity (%)'].rolling(window=window_size_main).mean()
df['SRH (%)_smooth'] = df['SRH'].rolling(window=window_size_main).mean()

# Create the figure and axes for the plots
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(4, 3))

# Plotting 'SRH (%)' (Surface Relative Humidity (%)) on y1 axis with smoothing and marker
ax1.plot(df.index/60, df['SRH (%)_smooth'], linestyle='-', color=colors['dark_blue'], label='SRH')
ax1.set_ylabel('Relative Humidity (\%)', color='black')

# Plotting 'Relative Humidity (%)' on y1 axis with smoothing and marker
ax1.plot(df.index/60, df['Relative Humidity (%)_smooth'], linestyle='-', color='purple', label='RH', alpha=0.5)
ax1.set_ylabel('Relative Humidity (\%)', color='black')

# Creating a secondary y-axis for temperature variables
ax2 = ax1.twinx()

# Plotting 'Temperature (°C)' (Air Temp) on y2 axis with smoothing and marker
ax2.plot(df.index/60, df['Temperature (°C)_smooth'], linestyle='-',  color='red', label=r"$T_{\text{surf}}$", alpha=0.5)
ax2.set_ylabel('Temperature (°C)', color='black')

# Plotting 'Surface Temp (°C)' on y2 axis with smoothing and marker
ax2.plot(df.index/60, df['Surface Temp (°C)_smooth'], linestyle='-', color='green', label=r"$T_{\text{air}}$", alpha=0.5)
ax2.set_ylabel('Temperature (°C)')

# Adding legends
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='lower right', fontsize=6)

# Setting x-axis label
ax1.set_xlabel('Elapsed Time (Minutes)')

ax1.set_xlim(left=0)
ax1.set_xlim(right=max(df.index/60))

# Adjust layout
fig.tight_layout()
plt.savefig('plot.png', dpi=300)

# Show plot
plt.show()


In [ ]:
# Plot for the paper where all the variables are shown but not the different shaded areas as it cloud the
# Import necessary libraries
import matplotlib.pyplot as plt
import scienceplots
import pandas as pd
from matplotlib.ticker import FuncFormatter
import numpy as np

def get_colors():
    return {
        'dark_blue': '#082a54',
        'purple': '#6a3577',  # Darker purple
        'teal': '#2b736a',    # Darker teal
        'gold': '#b89538',
        'red': '#e02b35',
        'dark_gray': '#1f77b4'  # Darker gray
    }

# Use custom style
plt.style.use('default')
plt.style.use(['science'])

df_nacl = pd.read_csv("path_to_csv.csv")

# Set 'Elapsed' as the index
df_nacl.set_index('Elapsed', inplace=True)

# Exponential smoothing span
span = 5

# Smooth the Average Droplet Radius with an exponential smoothing function
df_nacl['Average Droplet Radius (mm)'] = np.sqrt(df_nacl['Average Droplet Size (mm2)'] / np.pi)
df_nacl['Average Droplet Radius (mm)'] = df_nacl['Average Droplet Radius (mm)'].ewm(span=span, adjust=False).mean()

# Fill NaN values with the first value
df_nacl['Average Droplet Radius (mm)'].fillna(df_nacl['Average Droplet Radius (mm)'].iloc[0], inplace=True)

# Smooth the Surface Relative Humidity (SRH) with a rolling mean
rolling_window_size = 3
df_nacl['SRH'] = df_nacl['SRH'].rolling(window=rolling_window_size, min_periods=1).mean()

# Create 'Combined Conductivity' column if wanted to plot together potentially in a log scale
df_nacl['Combined Conductivity'] = np.nan

# Use 'Cond Lo Freq (µS)' where 'Cond Lo Freq (µS)' < 1 µS
mask_lo = df_nacl['Cond Lo Freq (µS) '] < 1
df_nacl.loc[mask_lo, 'Combined Conductivity'] = df_nacl.loc[mask_lo, 'Cond Lo Freq (µS) ']

# Find the index where 'Cond Hi Freq (µS)' first increases above 5 µS
first_hi_above_5_index = df_nacl[df_nacl['Cond Hi Freq (µS) '] > 5].index.min()

# If 'Cond Hi Freq (µS)' does go above 5 µS at some point
if pd.notna(first_hi_above_5_index):
    # Find the index where 'Cond Lo Freq (µS)' first reaches 1 µS
    first_lo_at_1_index = df_nacl[df_nacl['Cond Lo Freq (µS) '] == 1].index.min()
    
    # Ensure that both indices are valid
    if pd.notna(first_lo_at_1_index) and first_lo_at_1_index < first_hi_above_5_index:
        # Interpolate between 1 µS and the first 'Cond Hi Freq (µS)' value above 5 µS
        interpolate_indices = df_nacl.loc[first_lo_at_1_index:first_hi_above_5_index].index
        num_points = len(interpolate_indices)
        start_value = df_nacl.loc[first_lo_at_1_index, 'Cond Lo Freq (µS) ']
        end_value = df_nacl.loc[first_hi_above_5_index, 'Cond Hi Freq (µS) ']
        interpolated_values = np.linspace(start_value, end_value, num=num_points)
        df_nacl.loc[interpolate_indices, 'Combined Conductivity'] = interpolated_values

    # Use 'Cond Hi Freq (µS)' from the first index where it goes above 5 µS onward
    df_nacl.loc[first_hi_above_5_index:, 'Combined Conductivity'] = df_nacl.loc[first_hi_above_5_index:, 'Cond Hi Freq (µS) ']
else:
    # If 'Cond Hi Freq (µS)' never goes above 5 µS, interpolate up to 5 µS
    mask_interpolate = (df_nacl['Cond Lo Freq (µS) '] == 1) & (df_nacl['Cond Hi Freq (µS) '] == 5)
    df_nacl['interpolate_group'] = (mask_interpolate != mask_interpolate.shift(1)).cumsum() * mask_interpolate
    for group, df_group in df_nacl.groupby('interpolate_group'):
        if group == 0:
            continue
        num_points = len(df_group)
        interpolated_values = np.linspace(1, 5, num=num_points)
        df_nacl.loc[df_group.index, 'Combined Conductivity'] = interpolated_values
    df_nacl.drop('interpolate_group', axis=1, inplace=True)

# Ensure no missing values in 'Combined Conductivity' Solution Conductivity
df_nacl['Combined Conductivity'].ffill(inplace=True)

# Function to format x-axis as H:M
def format_time(seconds, pos):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    return f"{hours:02}:{minutes:02}"

# Get custom colors
colors = get_colors()

# Create the figure and axes (two subplots sharing the x-axis)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 5), sharex=True)

# Top subplot: Plot Free Corrosion Current
ax1.plot(df_nacl.index / 60, df_nacl['Free Corr (µA) '], linestyle='-', color=colors['teal'],
         markerfacecolor='none', label='Free Corrosion Current')
ax1.set_ylabel('Free Corrosion (µA)')
ax1.tick_params(axis='y')

ax1.tick_params(axis='y', which='both', colors=colors['teal'], labelcolor=colors['teal'])
ax1.yaxis.label.set_color(colors['teal'])
ax1.spines['left'].set_color(colors['teal'])

# Create a twin Axes sharing the x-axis for Surface Relative Humidity
ax1_twin1 = ax1.twinx()
ax1_twin1.plot(df_nacl.index / 60, df_nacl['SRH'], linestyle='-', color=colors['dark_blue'],
               markerfacecolor='none', label='SRH')
ax1_twin1.set_ylabel('Surface Relative Humidity (\%)')
ax1_twin1.tick_params(axis='y')

ax1_twin1.tick_params(axis='y', which='both', colors=colors['dark_blue'], labelcolor=colors['dark_blue'])
ax1_twin1.yaxis.label.set_color(colors['dark_blue'])
ax1_twin1.spines['right'].set_color(colors['dark_blue'])

# Create another twin Axes for Average Droplet Radius
ax1_twin2 = ax1.twinx()
ax1_twin2.spines["right"].set_position(("outward", 50))  # Offset the third y-axis to the right
ax1_twin2.plot(df_nacl.index / 60, df_nacl['Average Droplet Radius (mm)'], linestyle='-', color=colors['red'],
               markerfacecolor='none', label='Droplet Radius')
ax1_twin2.set_ylabel('Droplet Radius (mm)')
ax1_twin2.tick_params(axis='y')
ax1_twin2.spines["right"].set_visible(True)

# For ax1_twin2 (Average Droplet Radius - red)
ax1_twin2.tick_params(axis='y', which='both', colors=colors['red'], labelcolor=colors['red'])
ax1_twin2.yaxis.label.set_color(colors['red'])
ax1_twin2.spines['right'].set_color(colors['red'])

# Bottom subplot: Plot Galvanic Corrosion on the primary y-axis
ax2.plot(df_nacl.index / 60, df_nacl['Galv Corr (µA) '], linestyle='-', color=colors['dark_gray'],
         markerfacecolor='none', label='Galvanic Corrosion Current')
ax2.set_ylabel('Galvanic Corrosion (µA)')
ax2.tick_params(axis='y')

# For ax2 (Galvanic Corrosion Current - dark gray)
ax2.tick_params(axis='y', which='both', colors=colors['dark_gray'], labelcolor=colors['dark_gray'])
ax2.yaxis.label.set_color(colors['dark_gray'])
ax2.spines['left'].set_color(colors['dark_gray'])

# Create a twin Axes for 'Cond Lo Freq (µS)'
ax2_twin1 = ax2.twinx()
ax2_twin1.plot(df_nacl.index / 60, df_nacl['Cond Lo Freq (µS) '], linestyle='-', color=colors['gold'],
               markerfacecolor='none', label='LF Conductance')
ax2_twin1.set_ylabel('LF Conductance (µS)')
ax2_twin1.tick_params(axis='y')

# For ax2_twin1 (LF Conductance - gold)
ax2_twin1.tick_params(axis='y', which='both', colors=colors['gold'], labelcolor=colors['gold'])
ax2_twin1.yaxis.label.set_color(colors['gold'])
ax2_twin1.spines['right'].set_color(colors['gold'])

# Create another twin Axes for 'Cond Hi Freq (µS)'
ax2_twin2 = ax2.twinx()
ax2_twin2.spines["right"].set_position(("outward", 50))  # Offset the third y-axis to the right
ax2_twin2.plot(df_nacl.index / 60, df_nacl['Cond Hi Freq (µS) ']/1000, linestyle='-', color=colors['purple'],
               markerfacecolor='none', label='HF Conductance')
ax2_twin2.set_ylabel('HF Conductance (mS)')
ax2_twin2.tick_params(axis='y')
# Collect all handles and labels for legends
# ax2.xaxis.set_major_formatter(formatter)
ax2.set_xlabel('Elapsed Time (Minutes)')

# For ax2_twin2 (HF Conductance - purple)
ax2_twin2.tick_params(axis='y', which='both', colors=colors['purple'], labelcolor=colors['purple'])
ax2_twin2.yaxis.label.set_color(colors['purple'])
ax2_twin2.spines['right'].set_color(colors['purple'])

# Collect all handles and labels for legends
lines_labels_2 = [ax.get_legend_handles_labels() for ax in [ax2, ax2_twin1, ax2_twin2]]
lines2, labels2 = [sum(lol, []) for lol in zip(*lines_labels_2)]

# Collect all handles and labels for legends
lines_labels_1 = [ax.get_legend_handles_labels() for ax in [ax1, ax1_twin1, ax1_twin2]]
lines1, labels1 = [sum(lol, []) for lol in zip(*lines_labels_1)]

# Remove units from labels if necessary
labels2 = [label.split(' (')[0] for label in labels2]

# Add legends to each subplot in the upper-right corner
ax1.legend(lines1, labels1, loc='lower right', fontsize=6, frameon=True)
ax2.legend(lines2, labels2, loc='lower right', fontsize=6, frameon=True)

ax1.set_xlim(right=max(df.index/60))
ax1.set_ylim(bottom=0)
ax1_twin1.set_xlim(left=0)
ax1_twin1.set_ylim(bottom=84)
ax1_twin2.set_xlim(left=0)
ax1_twin2.set_ylim(bottom=0)
ax2.set_xlim(left=0)
ax2.set_ylim(bottom=0)
ax2_twin2.set_xlim(left=0)
ax2_twin2.set_ylim(bottom=0)
ax2_twin1.set_xlim(left=0)
ax2_twin1.set_ylim(bottom=0)


# Adjust layout to prevent overlap
fig.tight_layout()

# Save and show plot
plt.savefig('plot.png', dpi=300)
plt.show()

In [ ]:
# Following code generates 1 big plot with all variables
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from scipy.stats import pearsonr
from matplotlib.ticker import FormatStrFormatter


# Function to calculate Pearson correlation with lag
def calculate_pearson_corr(source, target, max_lag):
    pearson_values = [pearsonr(source, target)[0]]  # Lag zero
    for lag in range(1, max_lag + 1):
        pearson_value, _ = pearsonr(source[:-lag], target[lag:])
        pearson_values.append(pearson_value)
    return pearson_values

# Get consistent colors
def get_colors():
    return {
        'dark_blue': '#082a54',
        'purple': '#6a3577',
        'teal': '#2b736a',
        'gold': '#b89538',
        'red': '#e02b35',
        'gray': '#1f77b4'
    }

# Use custom style
plt.style.use('default')
plt.style.use(['science'])

# Load the CSV file
df = pd.read_csv("path_to_csv.csv")
df.set_index('Elapsed', inplace=True)

df['Free Corr (nA) '] = df['Free Corr (µA) '] * 1e3  # µA to nA
df['SRH (%)'] = df['SRH']
# Calculate the radius from the area, assuming areas are circles and rename the column
df['Droplet radius (mm)'] = np.sqrt(df['Average Droplet Size (mm2)'] / np.pi)

# Smooth the time series with a moving average
df['Droplet radius (mm)'] = df['Droplet radius (mm)'].ewm(span=8, adjust=False).mean()
df['Average Droplet Size (mm2)'] = df['Average Droplet Size (mm2)'].ewm(span=8, adjust=False).mean()

# Define the pairs of time series and the lags
pairs = [
    ('SRH (%)', 'Droplet radius (mm)'),
    ('Droplet radius (mm)', 'Free Corr (nA) '),
    ('Droplet radius (mm)', 'Cond Hi Freq (µS) '),
    ('Droplet radius (mm)', 'Cond Lo Freq (µS) '),
    ('Droplet radius (mm)', 'Galv Corr (µA) ')
]
max_lag = 55  # Maximum lag to consider

# Get the list of colors
color_list = get_colors()

# Fixed color map for specific variables
variable_colors = {
    'SRH (%)': color_list['dark_blue'],
    'Free Corr (nA) ': color_list['teal'],
    'Droplet radius (mm)': color_list['red'],
    'Cond Lo Freq (µS) ': color_list['purple'],
    'Cond Hi Freq (µS) ': color_list['gold'],
    'Galv Corr (µA) ': color_list['gray']
}

# Create a single figure with 5 rows and 2 columns
fig, axes = plt.subplots(5, 2, figsize=(9,9), gridspec_kw={'width_ratios': [2, 1]}, sharex='col')
fig.subplots_adjust(wspace=2, hspace=0.5)

# Function to plot the data
def plot_data(axes, pairs):
    for i, (source_name, target_name) in enumerate(pairs):
        try:
            source = df[source_name].dropna().values
            target = df[target_name].dropna().values

            min_length = min(len(source), len(target))
            source = source[:min_length]
            target = target[:min_length]

            lags = np.arange(-max_lag, max_lag + 1)
            source_label = source_name.split('(')[0].strip()
            target_label = target_name.split('(')[0].strip()

            # Pearson Correlation calculation
            dcor_values = calculate_pearson_corr(source, target, max_lag)
            max_corr_value = max(dcor_values)  # Max correlation value
            dcor_max_idx = dcor_values.index(max(dcor_values))

            # Print out the max correlation and its corresponding lag
            print(f'Max correlation for {source_label} -> {target_label}: {max_corr_value:.4f} at lag {dcor_max_idx} minutes')

            # Plot time series on the left
            ax2 = axes[i, 0]
            ax2.plot(df.index[:min_length] / 60, target, label=target_label, color=variable_colors[target_name], linewidth=1)
            ax2.set_ylabel(target_name, fontsize=11)
            ax1 = ax2.twinx()
            ax1.plot(df.index[:min_length] / 60, source, label=source_label, color=variable_colors[source_name], linewidth=1)
            # Ensure SRH (%) is set correctly for the first subplot
            if i == 0:  # First row, first column (top-left)
                ax1.set_ylabel("SRH (\%)", fontsize=11)
            else:
                ax1.set_ylabel(source_name, fontsize=11)
                
            ax1.tick_params(axis='both', which='major', labelsize=11)
            ax2.tick_params(axis='both', which='major', labelsize=11)
            ax1.set_xlim(left=0)
            ax1.set_xlim(right=max(df.index/60))
            # axes[i, 0].set_xlabel('Elapsed Time (minutes)', fontsize=13)
            # Add legends with frame
            lines, labels = ax1.get_legend_handles_labels()
            lines2, labels2 = ax2.get_legend_handles_labels()
            ax2.legend(lines + lines2, labels + labels2, frameon=True, loc='lower right', fontsize=8)

            # Plot Pearson Correlation on the right
            axes[i, 1].plot(range(max_lag + 1), dcor_values, color='black', linewidth=2)
            axes[i, 1].plot(dcor_max_idx, max_corr_value, 'o', color='darkgreen', markersize=7,
                            label=f'Max Corr: {round(max_corr_value, 2)} at Lag {dcor_max_idx} min')
            axes[i, 1].set_ylabel('Pearson Correlation', fontsize=11)
            axes[i, 1].legend(frameon=True, loc='lower right', fontsize=8)
            axes[i, 1].tick_params(axis='both', which='major', labelsize=11)
            axes[i, 1].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))
            axes[i, 1].set_xlim(left=0)
            axes[i, 1].set_xlim(right=max_lag)
            axes[i, 1].set_ylim(bottom=min(dcor_values) - 0.05, top=1)

        except Exception as e:
            print(f"Error processing pair ({source_name}, {target_name}): {e}")

# Plot data for all pairs
plot_data(axes, pairs)

# axes[i, 0].set_xlabel('Elapsed Time (minutes)', fontsize=13)
axes[-1, 0].set_xlabel('Elapsed Time (minutes)', fontsize=13)
axes[-1, 1].set_xlabel('Time Lags (minutes)', fontsize=13)

fig.tight_layout()
fig.savefig("plot.png", dpi=300)

plt.show()


In [ ]:
# For Both LUNA tests
# Evolution of the average droplet area and droplet radius over time for both LUNA 1M NaCl and demineralised water.
# Import necessary libraries
import matplotlib.pyplot as plt
import scienceplots
import pandas as pd
import matplotlib.dates as mdates
from matplotlib.ticker import FuncFormatter
import matplotlib.cm as cm
import numpy as np
import scienceplots
import scipy.stats as stats

# Use custom style
plt.style.use('default')
plt.style.use(['science'])

# Define the custom color palette
def get_colors():
    return {
        'dark_blue': '#082a54',
        'purple': '#6a3577',  # Darker purple
        'teal': '#2b736a',    # Darker teal
        'gold': '#b89538',
        'red': '#e02b35',
        'dark_gray': '#1f77b4'  # Darker gray
    }

# Load the datasets for pure water and 1M NaCl solution
df_distilled = pd.read_csv('path_to_csv')
df_nacl = pd.read_csv('path_to_csv')

# Set the confidence interval parameters
confidence = 0.95
n = 3
t_value = stats.t.ppf((1 + confidence) / 2., n - 1)

# Define the area columns
area_columns = ['Average Droplet Size (mm2)', 'Average Droplet Size (mm2)_2', 'Average Droplet Size (mm2)_3']

# For df_distilled
# Compute mean, SE, and CI for area
df_distilled['Average Droplet Size Mean (mm2)'] = df_distilled[area_columns].mean(axis=1)
df_distilled['Average Droplet Size SE (mm2)'] = df_distilled[area_columns].sem(axis=1)
df_distilled['Average Droplet Size CI (mm2)'] = t_value * df_distilled['Average Droplet Size SE (mm2)']

# Compute radius for each area column
for col in area_columns:
    radius_col = col.replace('Average Droplet Size', 'Average Droplet Radius')
    df_distilled[radius_col] = np.sqrt(df_distilled[col] / np.pi)

# Define the radius columns
radius_columns = [col.replace('Average Droplet Size', 'Average Droplet Radius') for col in area_columns]

# Compute mean, SE, and CI for radius
df_distilled['Average Droplet Radius Mean (mm)'] = df_distilled[radius_columns].mean(axis=1)
df_distilled['Average Droplet Radius SE (mm)'] = df_distilled[radius_columns].sem(axis=1)
df_distilled['Average Droplet Radius CI (mm)'] = t_value * df_distilled['Average Droplet Radius SE (mm)']

# Apply exponential smoothing to the mean droplet radius
span = 5
df_distilled['Average Droplet Radius Mean (mm)'] = df_distilled['Average Droplet Radius Mean (mm)'].ewm(span=span, adjust=False).mean()

# Repeat the same steps for df_nacl
df_nacl['Average Droplet Size Mean (mm2)'] = df_nacl[area_columns].mean(axis=1)
df_nacl['Average Droplet Size SE (mm2)'] = df_nacl[area_columns].sem(axis=1)
df_nacl['Average Droplet Size CI (mm2)'] = t_value * df_nacl['Average Droplet Size SE (mm2)']

for col in area_columns:
    radius_col = col.replace('Average Droplet Size', 'Average Droplet Radius')
    df_nacl[radius_col] = np.sqrt(df_nacl[col] / np.pi)

df_nacl['Average Droplet Radius Mean (mm)'] = df_nacl[radius_columns].mean(axis=1)
df_nacl['Average Droplet Radius SE (mm)'] = df_nacl[radius_columns].sem(axis=1)
df_nacl['Average Droplet Radius CI (mm)'] = t_value * df_nacl['Average Droplet Radius SE (mm)']

# Apply exponential smoothing to the mean droplet radius
df_nacl['Average Droplet Radius Mean (mm)'] = df_nacl['Average Droplet Radius Mean (mm)'].ewm(span=span, adjust=False).mean()

# Convert 'Elapsed' to minutes
df_distilled['Elapsed_minutes'] = df_distilled['Elapsed'] / 60
df_nacl['Elapsed_minutes'] = df_nacl['Elapsed'] / 60

# Set the index as 'Elapsed_minutes' time for both dataframes
df_distilled.set_index('Elapsed_minutes', inplace=True)
df_nacl.set_index('Elapsed_minutes', inplace=True)

# Trim df_nacl to the length of df_distilled
max_time = df_distilled.index.max()
df_nacl = df_nacl[df_nacl.index <= max_time]

# Define the colors
colors = get_colors()

# Map colors to datasets
dataset_styles = {
    'Pure Water': {'color': colors['dark_blue']},
    '1M NaCl Solution': {'color': colors['red']}
}

# Create the side-by-side subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 3))
plt.subplots_adjust(wspace=0.4)  # default is around 0.2

# Plot the average surface area with confidence intervals (left subplot)
ax1.plot(df_distilled.index, df_distilled['Average Droplet Size Mean (mm2)'],
         label='Demineralized Water', color=dataset_styles['Pure Water']['color'],
         linestyle='-', linewidth=1)
ax1.fill_between(df_distilled.index,
                 df_distilled['Average Droplet Size Mean (mm2)'] - df_distilled['Average Droplet Size CI (mm2)'],
                 df_distilled['Average Droplet Size Mean (mm2)'] + df_distilled['Average Droplet Size CI (mm2)'],
                 color=dataset_styles['Pure Water']['color'], alpha=0.3)

ax1.plot(df_nacl.index, df_nacl['Average Droplet Size Mean (mm2)'],
         label='1M NaCl', color=dataset_styles['1M NaCl Solution']['color'],
         linestyle='-', linewidth=1)
ax1.fill_between(df_nacl.index,
                 df_nacl['Average Droplet Size Mean (mm2)'] - df_nacl['Average Droplet Size CI (mm2)'],
                 df_nacl['Average Droplet Size Mean (mm2)'] + df_nacl['Average Droplet Size CI (mm2)'],
                 color=dataset_styles['1M NaCl Solution']['color'], alpha=0.3)

ax1.set_ylabel('Average Droplet Area (mm²)')
ax1.set_xlabel('Elapsed Time (Minutes)')
ax1.legend(loc='lower right', fontsize=6)

# Plot the smoothed average droplet radius with confidence intervals (right subplot)
ax2.plot(df_distilled.index, df_distilled['Average Droplet Radius Mean (mm)'],
         label='Demineralized Water (smoothed)', color=dataset_styles['Pure Water']['color'],
         linestyle='-', linewidth=1)
ax2.fill_between(df_distilled.index,
                 df_distilled['Average Droplet Radius Mean (mm)'] - df_distilled['Average Droplet Radius CI (mm)'],
                 df_distilled['Average Droplet Radius Mean (mm)'] + df_distilled['Average Droplet Radius CI (mm)'],
                 color=dataset_styles['Pure Water']['color'], alpha=0.3)

ax2.plot(df_nacl.index, df_nacl['Average Droplet Radius Mean (mm)'],
         label='1M NaCl (smoothed)', color=dataset_styles['1M NaCl Solution']['color'],
         linestyle='-', linewidth=1)
ax2.fill_between(df_nacl.index,
                 df_nacl['Average Droplet Radius Mean (mm)'] - df_nacl['Average Droplet Radius CI (mm)'],
                 df_nacl['Average Droplet Radius Mean (mm)'] + df_nacl['Average Droplet Radius CI (mm)'],
                 color=dataset_styles['1M NaCl Solution']['color'], alpha=0.3)

ax2.set_xlabel('Elapsed Time (Minutes)')
ax2.set_ylabel('Average Droplet Radius (mm)')
ax2.legend(loc='lower right', fontsize=6)

ax1.set_xlim(left=0)
ax1.set_xlim(right=max(df_distilled.index))
ax1.set_ylim(bottom=0)
ax1.set_ylim(top=0.3)
ax2.set_xlim(left=0)
ax2.set_xlim(right=max(df_distilled.index))
ax2.set_ylim(bottom=0)
ax2.set_ylim(top=0.3)

# Adjust layout and show plot
fig.tight_layout()
fig.subplots_adjust(wspace=0.35)  # try values between 0.3 and 1.0
plt.savefig('plot.png', dpi=300)
plt.show()